In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sm
import random

In [2]:
def deljustz(proj, z):
    ''' Removes consecutive repeated points that in the 3D knot were just a change in the z coordinate '''
    
    extra = []
    for i in range(len(proj)-1):
        if all(proj[i] == proj[i+1]):
            extra.append(i+1) #If two consecutive points are the same, the second will be deleted
    
    proj = np.delete(proj, extra, axis=0)
    z = np.delete(z, extra)
    return proj, z

In [3]:
def crossings(proj, z):
    over = np.array([])
    under = np.array([])
    for i in range(1,len(proj)):
        for j in range(i+1,len(proj)): #Check for repetition only in the points after so the same crossing isn't checked twice
            if all(proj[i] == proj[j]):
                if z[i] > z[j]:
                    over = np.append(over,i)
                    under = np.append(under,j)
                else:
                    over = np.append(over,j)
                    under = np.append(under,i)
    
    over = over[np.argsort(under)] #Sorts overcrossings with their corresponding undercrossing (sorted below)
    under = np.sort(under) #Sorts undercrossings in order of first to last
    return under, over

In [4]:
def alex(x,y,z):
    ''' Assumes every point of the knot lies on an integer 3D coordinate '''
    ''' Assumes you have the x,y,z coordinates for the curve '''
    ''' Assumes there are atleast two crossings in the knot '''
    ''' Assumes there are no overlapping line segments in the projection '''
    ''' Assumes the start and end point of the curve is not a crossing in the projection '''
    ''' Assumes that no two consecutive undercrossings are also consecutive points in the projection '''
    
    proj_ = np.array([x,y]).T
    proj, zn = deljustz(proj_, z)
    us, os = crossings(proj, zn)
    ulen = len(us)
    ubins = np.array([0] + us.tolist() + [len(proj)-1])
    #Demarcates the arcs into bins with the endpoints being the undercrossings
    #The undercrossing itself is placed into the bin (arc) it is the left edge of
    
    t = sm.symbols('t')
    alexvars = [1, t, -t, -1]
    alexmat = sm.zeros(ulen)
    
    for i in range(ulen):
        vec1 = proj[int(os[i]+1)] - proj[int(os[i])]
        vec2 = proj[int(us[i]+1)] - proj[int(us[i])]
        if np.cross(vec1,vec2) == -1: #Point after overcrossing --> Point after undercrossing : Clockwise
            ps = [os[i], us[i]+1, os[i], us[i]-1]
        else:  #Point after overcrossing --> Point after undercrossing : Anti-clockwise
            ps = [os[i], us[i]-1, os[i], us[i]+1]
            
        # Though the Alexander Polynomial in actuality checks the arcs immediately before and after the crossing,
        # not the point of the crossing itself, the overcrossing point must lie on the same arc as its two neighbours
        # so to prevent errors from the neighbours being undercrossings (as in Trefoil.csv), I have used the
        # points themselves.
        
        for j in range(4):
            arcbin,_ = np.histogram(ps[j], bins=ubins)
            arc = np.argmax(arcbin)%ulen #Which arc is the point in, with the first and last bin being the same arc
            alexmat[ulen*i + arc] += alexvars[j]
    
    alexmat.col_del(0)
    alexmat.row_del(0)
    return alexmat.det()

In [5]:
x,y,z = np.loadtxt('Trefoil.csv', delimiter=',', unpack=True)
alex(x,y,z)

t**2 - t + 1

In [6]:
x_,y_,z_ = np.loadtxt('Figure 8.csv', delimiter=',', unpack=True)
alex(x_,y_,z_)

t**2 - 3*t + 1